In [19]:
import geopandas
from shapely import Polygon
import subprocess


import os
import json
import wget
import zipfile


from dotenv import load_dotenv

from openai import OpenAI

In [12]:
load_dotenv("../.env")

output_directory_base = "../data/sitios_prioritarios"
name = "Sitios_Prioritarios"

# Load files
df = geopandas.read_file(
    os.path.join(output_directory_base, name + ".shp")
)

In [13]:
df_demo = geopandas.read_file(
    "../data/demo_data/doc.kml"
)

In [14]:
df_kms = df.to_crs("EPSG:32633")
demo_geometry_kms = df_demo.to_crs("EPSG:32633")

In [15]:
class PrioritySitesAssistant:

    def __init__(self, df: geopandas.GeoDataFrame, client: OpenAI):
        self.df = df
        self.client = client

    def system_prompt(self) -> str:        
        system_prompt = """
        Eres un experto en sitios prioritarios de alta importancia para la conservación ambiental, protección de la biodiversidad, 
        o preservación de ciertos recursos naturales o culturales y te han contratado para hacer la evaluación de impacto ambiental
        de un nuevo proyecto de parque fotovoltaico en Chile bajo su respectiva legislación.

        El usuario entregará su locacion y los sitios prioritarios cercanos en un radio de 3km.

        Tu trabajo es entregar una detallada evaluación de impacto ambiental del proyecto fotovoltaico en los sitios prioritarios cercanos.

        Para esto primero vas a entregar un resumen si encuentras algo critico, para rechazar el proyecto, si no di que todo esta bien.

        En caso de que encuentres algo critico, debes entregar una evaluación detallada de los impactos ambientales y sociales del proyecto.

        El output será un JSON con dos campos
        ```json
        {
            "resumen": Resumen breve de la evaluacion,
            "evaluacion": Detalle de la evaluacion
        }
        ```
        """
        return system_prompt
    
    def format_message(self, close_sites: geopandas.GeoDataFrame) -> str:
        close_sites_data = str(close_sites.to_dict("records"))
        message = """
        Los sitios prioritarios cercanos a menos de 3km son:
        {close_sites}
        """
        return message.replace("{close_sites}", close_sites_data)


    def evaluate_project(self, location: Polygon, threshold_in_meters: int = 3000) -> dict:
        distance = self.df.geometry.distance(location.iloc[0].geometry)

        close_parks = self.df[distance < threshold_in_meters]
        
        print("Numero de sitios cercanos: ", close_parks.shape[0])

        system_prompt = self.system_prompt()
        message = self.format_message(close_parks)

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": message}
        ]
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            response_format={ "type": "json_object" },
            temperature=0
        )
        return json.loads(response.choices[0].message.content)

In [16]:
ps_assistant = PrioritySitesAssistant(df_kms, OpenAI())

In [17]:
response = ps_assistant.evaluate_project(demo_geometry_kms, threshold_in_meters=3000)

Numero de sitios cercanos:  1


In [18]:
response

{'resumen': 'Se encontró que el proyecto de parque fotovoltaico impactaría negativamente en el Estero Casablanca, un sitio prioritario de alta importancia para la conservación ambiental.',
 'evaluacion': 'El proyecto de parque fotovoltaico en las cercanías del Estero Casablanca podría tener impactos ambientales significativos, como la fragmentación del hábitat de especies nativas, la contaminación del agua y la alteración de los flujos hidrológicos. Además, la construcción y operación del parque podrían generar ruidos y alteraciones visuales que afecten negativamente a la fauna y flora del estero. También se debe considerar el posible impacto social en las comunidades locales que dependen de este recurso hídrico para sus actividades económicas y culturales. Se recomienda realizar estudios más detallados y medidas de mitigación antes de aprobar el proyecto.'}